In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

csv_content = """1,Mickéy Mousé,56,70kgs,72,69,71,-,-,-
2,Donald Duck,34,154.89lbs,-,-,-,85,84,76
3,Mini Mouse,16,,-,-,-,65,69,72
4,Scrooge McDuck,,78kgs,78,79,72,-,-,-
5,Pink Panther,54,198.658lbs,-,-,-,69,,75
6,Huey McDuck,52,189lbs,-,-,-,68,75,72
7,Dewey McDuck,19,56kgs,-,-,-,71,78,75
8,Scööpy Doo,32,78kgs,78,76,75,-,-,-
,,,,,,,,,
,,,,,,,,,
9,Huey McDuck,52,189lbs,-,-,-,68,75,72
10,Louie McDuck,12,45kgs,-,-,-,92,95,87
11,Henry Nam,,60kgs,78,75,72,-,-,-
12,Michel Long,34,,65,67,55,-,-,-
13,Tana Ricky,,,-,-,-,68,72,70
14,UniLever,,,,,,,,
15,,52,180lbs,-,-,-,68,75,72"""

with open('patient_heart_rate.csv', 'w', encoding='utf-8') as f:
    f.write(csv_content)

print("Đã tạo file patient_heart_rate.csv thành công.\n")

Đã tạo file patient_heart_rate.csv thành công.



# 1. Xử lý thiếu Header

In [37]:
column_names = ["Id", "Name", "Age", "Weight", "m0006", "m0612", "m1218", "f0006", "f0612", "f1218"]
df = pd.read_csv('patient_heart_rate.csv', names=column_names, encoding='utf-8', na_values=['-', ''])
print(df.head())

    Id            Name   Age      Weight  m0006  m0612  m1218  f0006  f0612  f1218
0  1.0    Mickéy Mousé  56.0       70kgs   72.0   69.0   71.0    NaN    NaN    NaN
1  2.0     Donald Duck  34.0   154.89lbs    NaN    NaN    NaN   85.0   84.0   76.0
2  3.0      Mini Mouse  16.0         NaN    NaN    NaN    NaN   65.0   69.0   72.0
3  4.0  Scrooge McDuck   NaN       78kgs   78.0   79.0   72.0    NaN    NaN    NaN
4  5.0    Pink Panther  54.0  198.658lbs    NaN    NaN    NaN   69.0    NaN   75.0


# 2. Tách cột Name


In [39]:
df[['Firstname', 'Lastname']] = df['Name'].str.split(expand=True)
df = df.drop('Name', axis=1) # [cite: 264]
print(df.head())

    Id   Age      Weight  m0006  m0612  m1218  f0006  f0612  f1218 Firstname Lastname
0  1.0  56.0       70kgs   72.0   69.0   71.0    NaN    NaN    NaN    Mickéy    Mousé
1  2.0  34.0   154.89lbs    NaN    NaN    NaN   85.0   84.0   76.0    Donald     Duck
2  3.0  16.0         NaN    NaN    NaN    NaN   65.0   69.0   72.0      Mini    Mouse
3  4.0   NaN       78kgs   78.0   79.0   72.0    NaN    NaN    NaN   Scrooge   McDuck
4  5.0  54.0  198.658lbs    NaN    NaN    NaN   69.0    NaN   75.0      Pink  Panther


# 3. Chuẩn hóa cột Weight

In [40]:
def clean_weight(value):
    if pd.isna(value):
        return value
    value = str(value)
    if 'lbs' in value:
        weight = float(value.replace('lbs', '')) / 2.2
        return int(weight)
    elif 'kgs' in value:
        return int(float(value.replace('kgs', '')))
    return value

df['Weight'] = df['Weight'].apply(clean_weight)
print(df.head())

    Id   Age  Weight  m0006  m0612  m1218  f0006  f0612  f1218 Firstname Lastname
0  1.0  56.0    70.0   72.0   69.0   71.0    NaN    NaN    NaN    Mickéy    Mousé
1  2.0  34.0    70.0    NaN    NaN    NaN   85.0   84.0   76.0    Donald     Duck
2  3.0  16.0     NaN    NaN    NaN    NaN   65.0   69.0   72.0      Mini    Mouse
3  4.0   NaN    78.0   78.0   79.0   72.0    NaN    NaN    NaN   Scrooge   McDuck
4  5.0  54.0    90.0    NaN    NaN    NaN   69.0    NaN   75.0      Pink  Panther


# 4. Xóa dòng rỗng

In [41]:
df.dropna(how="all", inplace=True)
print(df)

      Id   Age  Weight  m0006  m0612  m1218  f0006  f0612  f1218 Firstname Lastname
0    1.0  56.0    70.0   72.0   69.0   71.0    NaN    NaN    NaN    Mickéy    Mousé
1    2.0  34.0    70.0    NaN    NaN    NaN   85.0   84.0   76.0    Donald     Duck
2    3.0  16.0     NaN    NaN    NaN    NaN   65.0   69.0   72.0      Mini    Mouse
3    4.0   NaN    78.0   78.0   79.0   72.0    NaN    NaN    NaN   Scrooge   McDuck
4    5.0  54.0    90.0    NaN    NaN    NaN   69.0    NaN   75.0      Pink  Panther
5    6.0  52.0    85.0    NaN    NaN    NaN   68.0   75.0   72.0      Huey   McDuck
6    7.0  19.0    56.0    NaN    NaN    NaN   71.0   78.0   75.0     Dewey   McDuck
7    8.0  32.0    78.0   78.0   76.0   75.0    NaN    NaN    NaN    Scööpy      Doo
10   9.0  52.0    85.0    NaN    NaN    NaN   68.0   75.0   72.0      Huey   McDuck
11  10.0  12.0    45.0    NaN    NaN    NaN   92.0   95.0   87.0     Louie   McDuck
12  11.0   NaN    60.0   78.0   75.0   72.0    NaN    NaN    NaN     Henry  

# 5. Xử lý dữ liệu trùng lặp

In [42]:
df.drop_duplicates(subset=['Firstname', 'Lastname', 'Age', 'Weight'], keep='first', inplace=True)
print(df)

      Id   Age  Weight  m0006  m0612  m1218  f0006  f0612  f1218 Firstname Lastname
0    1.0  56.0    70.0   72.0   69.0   71.0    NaN    NaN    NaN    Mickéy    Mousé
1    2.0  34.0    70.0    NaN    NaN    NaN   85.0   84.0   76.0    Donald     Duck
2    3.0  16.0     NaN    NaN    NaN    NaN   65.0   69.0   72.0      Mini    Mouse
3    4.0   NaN    78.0   78.0   79.0   72.0    NaN    NaN    NaN   Scrooge   McDuck
4    5.0  54.0    90.0    NaN    NaN    NaN   69.0    NaN   75.0      Pink  Panther
5    6.0  52.0    85.0    NaN    NaN    NaN   68.0   75.0   72.0      Huey   McDuck
6    7.0  19.0    56.0    NaN    NaN    NaN   71.0   78.0   75.0     Dewey   McDuck
7    8.0  32.0    78.0   78.0   76.0   75.0    NaN    NaN    NaN    Scööpy      Doo
11  10.0  12.0    45.0    NaN    NaN    NaN   92.0   95.0   87.0     Louie   McDuck
12  11.0   NaN    60.0   78.0   75.0   72.0    NaN    NaN    NaN     Henry      Nam
13  12.0  34.0     NaN   65.0   67.0   55.0    NaN    NaN    NaN    Michel  

# 6. Xử lý lỗi Non-ASCII

In [47]:
df['Firstname'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
df['Lastname'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
print(df)

      Id   Age  Weight  m0006  m0612  m1218  f0006  f0612  f1218 Firstname Lastname
0    1.0  56.0    70.0   72.0   69.0   71.0    NaN    NaN    NaN     Micky     Mous
1    2.0  34.0    70.0    NaN    NaN    NaN   85.0   84.0   76.0    Donald     Duck
2    3.0  16.0    71.3    NaN    NaN    NaN   65.0   69.0   72.0      Mini    Mouse
3    4.0  36.1    78.0   78.0   79.0   72.0    NaN    NaN    NaN   Scrooge   McDuck
4    5.0  54.0    90.0    NaN    NaN    NaN   69.0    NaN   75.0      Pink  Panther
5    6.0  52.0    85.0    NaN    NaN    NaN   68.0   75.0   72.0      Huey   McDuck
6    7.0  19.0    56.0    NaN    NaN    NaN   71.0   78.0   75.0     Dewey   McDuck
7    8.0  32.0    78.0   78.0   76.0   75.0    NaN    NaN    NaN      Scpy      Doo
11  10.0  12.0    45.0    NaN    NaN    NaN   92.0   95.0   87.0     Louie   McDuck
12  11.0  36.1    60.0   78.0   75.0   72.0    NaN    NaN    NaN     Henry      Nam
13  12.0  34.0    71.3   65.0   67.0   55.0    NaN    NaN    NaN    Michel  

# 7. Xử lý Missing Values (Age, Weight)

In [48]:
print("Missing values before fix:")
print(df[['Age', 'Weight']].isnull().sum())
df['Age'] = df['Age'].astype(float)
df['Weight'] = df['Weight'].astype(float)
df['Age'].fillna(df['Age'].mean(), inplace=True)
df['Weight'].fillna(df['Weight'].mean(), inplace=True)

print("\nData after filling missing values:")
print(df)

Missing values before fix:
Age       0
Weight    0
dtype: int64

Data after filling missing values:
      Id   Age  Weight  m0006  m0612  m1218  f0006  f0612  f1218 Firstname Lastname
0    1.0  56.0    70.0   72.0   69.0   71.0    NaN    NaN    NaN     Micky     Mous
1    2.0  34.0    70.0    NaN    NaN    NaN   85.0   84.0   76.0    Donald     Duck
2    3.0  16.0    71.3    NaN    NaN    NaN   65.0   69.0   72.0      Mini    Mouse
3    4.0  36.1    78.0   78.0   79.0   72.0    NaN    NaN    NaN   Scrooge   McDuck
4    5.0  54.0    90.0    NaN    NaN    NaN   69.0    NaN   75.0      Pink  Panther
5    6.0  52.0    85.0    NaN    NaN    NaN   68.0   75.0   72.0      Huey   McDuck
6    7.0  19.0    56.0    NaN    NaN    NaN   71.0   78.0   75.0     Dewey   McDuck
7    8.0  32.0    78.0   78.0   76.0   75.0    NaN    NaN    NaN      Scpy      Doo
11  10.0  12.0    45.0    NaN    NaN    NaN   92.0   95.0   87.0     Louie   McDuck
12  11.0  36.1    60.0   78.0   75.0   72.0    NaN    NaN   

# 8. Melt & Split Columns (Tách cột Sex và Time)

In [49]:
df = pd.melt(df, id_vars=['Id', 'Age', 'Weight', 'Firstname', 'Lastname'], value_name="PulseRate", var_name="sex_and_time")

tmp_df = df["sex_and_time"].str.extract(r"(\D)(\d{2})(\d{2})", expand=True)
tmp_df.columns = ["Sex", "hours_lower", "hours_upper"]
tmp_df["Time"] = tmp_df["hours_lower"] + "-" + tmp_df["hours_upper"]

df = pd.concat([df, tmp_df], axis=1)
df = df.drop(['sex_and_time', 'hours_lower', 'hours_upper'], axis=1)
df = df.dropna(subset=['Id', 'Age', 'Weight', 'Firstname', 'Lastname', 'Sex', 'Time'])
df = df.sort_values(['Id', 'Time']).reset_index(drop=True)

print(df.head(15))

     Id   Age  Weight Firstname Lastname  PulseRate Sex   Time
0   1.0  56.0    70.0     Micky     Mous       72.0   m  00-06
1   1.0  56.0    70.0     Micky     Mous        NaN   f  00-06
2   1.0  56.0    70.0     Micky     Mous       69.0   m  06-12
3   1.0  56.0    70.0     Micky     Mous        NaN   f  06-12
4   1.0  56.0    70.0     Micky     Mous       71.0   m  12-18
5   1.0  56.0    70.0     Micky     Mous        NaN   f  12-18
6   2.0  34.0    70.0    Donald     Duck        NaN   m  00-06
7   2.0  34.0    70.0    Donald     Duck       85.0   f  00-06
8   2.0  34.0    70.0    Donald     Duck        NaN   m  06-12
9   2.0  34.0    70.0    Donald     Duck       84.0   f  06-12
10  2.0  34.0    70.0    Donald     Duck        NaN   m  12-18
11  2.0  34.0    70.0    Donald     Duck       76.0   f  12-18
12  3.0  16.0    71.3      Mini    Mouse        NaN   m  00-06
13  3.0  16.0    71.3      Mini    Mouse       65.0   f  00-06
14  3.0  16.0    71.3      Mini    Mouse        NaN   m

# 11. Xử lý Missing Values cho PulseRate


In [51]:
df['PulseRate'] = pd.to_numeric(df['PulseRate'], errors='coerce')

print("Missing PulseRate before fix:", df['PulseRate'].isnull().sum())
def fill_pulse_rate(group):
    group['PulseRate'] = group['PulseRate'].interpolate(method='linear', limit_direction='both')
    if group['PulseRate'].isnull().any():
        group['PulseRate'] = group['PulseRate'].fillna(group['PulseRate'].mean())
    return group

df = df.groupby('Id', group_keys=False).apply(fill_pulse_rate)

df['PulseRate'] = df['PulseRate'].fillna(df.groupby('Sex')['PulseRate'].transform('mean'))
df['PulseRate'] = df['PulseRate'].fillna(df['PulseRate'].mean())

print("Missing PulseRate after fix:", df['PulseRate'].isnull().sum())
print(df.head(10))

Missing PulseRate before fix: 37
Missing PulseRate after fix: 0
    Id   Age  Weight Firstname Lastname  PulseRate Sex   Time
0  1.0  56.0    70.0     Micky     Mous       72.0   m  00-06
1  1.0  56.0    70.0     Micky     Mous       70.5   f  00-06
2  1.0  56.0    70.0     Micky     Mous       69.0   m  06-12
3  1.0  56.0    70.0     Micky     Mous       70.0   f  06-12
4  1.0  56.0    70.0     Micky     Mous       71.0   m  12-18
5  1.0  56.0    70.0     Micky     Mous       71.0   f  12-18
6  2.0  34.0    70.0    Donald     Duck       85.0   m  00-06
7  2.0  34.0    70.0    Donald     Duck       85.0   f  00-06
8  2.0  34.0    70.0    Donald     Duck       84.5   m  06-12
9  2.0  34.0    70.0    Donald     Duck       84.0   f  06-12


# 12. Lưu trữ dữ liệu

In [52]:
df = df[["Id", "Firstname", "Lastname", "Age", "Weight", "Sex", "Time", "PulseRate"]]

output_filename = 'patient_heart_rate_clean.csv'
df.to_csv(output_filename, index=False)
print(f"Data saved to {output_filename}")
print(df.head())

Data saved to patient_heart_rate_clean.csv
    Id Firstname Lastname   Age  Weight Sex   Time  PulseRate
0  1.0     Micky     Mous  56.0    70.0   m  00-06       72.0
1  1.0     Micky     Mous  56.0    70.0   f  00-06       70.5
2  1.0     Micky     Mous  56.0    70.0   m  06-12       69.0
3  1.0     Micky     Mous  56.0    70.0   f  06-12       70.0
4  1.0     Micky     Mous  56.0    70.0   m  12-18       71.0
